# Récupération et traitement des données

In [111]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import pynsee as yns
import requests
from cartiflette.s3 import download_vectorfile_url_all

In [20]:
from pathlib import Path

In [21]:
Path.cwd()

PosixPath('/home/cathu/Documents/ENSAE/Projet_Catherine_Christelle')

## Les aménagements cyclables en Ile de France

In [63]:
a_velo= gpd.read_file('amenagements-velo-en-ile-de-france.geojson')

In [65]:
# Nombre d'entrées
a_velo.shape[0]

112217

In [66]:
a_velo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [67]:
a_velo['longueur'].describe()

count    112217.000000
mean        126.315959
std         174.519817
min           0.000000
25%          24.000000
50%          70.000000
75%         161.000000
max        3984.000000
Name: longueur, dtype: float64

In [68]:
a_velo.sample(n=1)

,osm_id,nom_com,sens_voit,ag,panneaux,moyenn_ech,revetement,highway,insee_com,nom_voie,longueur,petite_ech,nv,ad,geometry
27659,33734103.0,Clamart,DOUBLE,None,None,32,asphalt,residential,92023,Allée du Grand Cèdre,81,3,limite 30,None,"LINESTRING (2.26431 48.79830, 2.26427 48.79825..."


Pour le moment, nous travaillons sans les données de géolocalisation qui ne sont pas nécessaires aux calculs. Nous créons pour cela une nouvelle dataframe

In [69]:
col_to_keep = ["nom_com", "sens_voit", "ag", "panneaux", "revetement", "highway", "insee_com", "longueur", "nv", "ad"] 

In [70]:
# df_a_velo = a_velo[col_to_keep].copy()

df_a_velo = a_velo

print(df_a_velo.head())

         osm_id            nom_com sens_voit                              ag  \
0  4.014620e+08            Chelles    DOUBLE                            None   
1  4.048666e+08   La Queue-en-Brie        NC  chemin service site propre uni   
2  1.154304e+09            Lésigny        NC                  voie verte uni   
3  3.300503e+08  Pontault-Combault    UNIQUE                       DSC bande   
4  1.104033e+09   Champs-sur-Marne    UNIQUE                            None   

  panneaux moyenn_ech revetement      highway insee_com  \
0     None         32       None      service     77108   
1     None         11       None        track     94060   
2     None         11    asphalt         path     77249   
3     None         22       None  residential     77373   
4     None         22    asphalt  residential     77083   

                     nom_voie  longueur petite_ech           nv  \
0  Rue de la Mare Longue Noue        24          3    limite 30   
1             Allée Jacquette 

### Création de variables d'étude pour les aménagements cyclables

Nous souhaitons étudier la répartition géograhique des aménagements cyclables et particulièrement leur densité en mètre par habitants. Toutefois, l'ensemble des pistes cyclables n'est pas de la même "qualité" : séparée de la route ou non, sens inverse de la circulation, type de revêtement, etc. Pour prendre en compte la qualité des aménagements cyclables, nous pouvons donner des coefficients aux mètres de pistes selon ces différents critères lors du calcul du nombre total de mètres aménagés par commune. Nous allons proposer plusieurs méthodes de calcul selon ces critères.

In [71]:
print(df_a_velo["sens_voit"].unique())
print(df_a_velo["ag"].unique())
print(df_a_velo["revetement"].unique())
print(df_a_velo["highway"].unique())
print(df_a_velo["nv"].unique())
print(df_a_velo["ad"].unique())

['DOUBLE' 'NC' 'UNIQUE']
[None 'chemin service site propre uni' 'voie verte uni' 'DSC bande'
 'autre chemin velo uni' 'piste uni' 'DSC' 'piste trottoir uni'
 'bande uni' 'cheminement trottoir uni' 'goulotte' 'chemin dedie uni'
 'chaucidou' 'cheminement uni' 'DSC piste' 'voie bus uni' 'piste bi'
 'bande bi' 'shoulder uni']
[None 'asphalt' 'compacted' 'concrete' 'unpaved' 'wood' 'paving_stones'
 'ground' 'sett' 'gravel' 'sand' 'paved' 'fine_gravel' 'dirt'
 'cobblestone' 'concrete:plates' 'concrete:lanes' 'grass'
 'cobblestone:flattened' 'metal' 'pebblestone' 'unhewn_cobblestone'
 'bricks' 'tartan' 'pavés' 'vegecol' 'artificial_turf' 'grass_paver'
 'earth' 'bitume' 'à_définir' 'rock']
['service' 'track' 'path' 'residential' 'footway' 'cycleway' 'secondary'
 'tertiary' 'living_street' 'unclassified' 'pedestrian' 'steps' 'primary'
 'primary_link' 'secondary_link' 'tertiary_link' 'trunk_link'
 'motorway_link']
['limite 30' 'hors voirie' None 'z20' 'rue pietonne' 'z30' 'escalier velo'
 'velor

In [72]:
df_a_velo['sens_voit'].isna().sum()

0

In [73]:
df_a_velo['ag'].isna().sum()

68312

In [74]:
df_a_velo['revetement'].isna().sum()

38711

In [75]:
df_a_velo['highway'].isna().sum()

0

In [76]:
df_a_velo['nv'].isna().sum()

9548

In [77]:
df_a_velo['ad'].isna().sum()

68522

Nous n'allons donc garder que le critères suivant :  type de route ("highway", cf [documentation OpenstreeMap](https://wiki.openstreetmap.org/wiki/Key:highway) ). Les autres variables ont soit trop de valeurs manquantes soit manquent d'intérêt seule (sens des voitures). Nous allons produire 2 variables : l'une pondérée avec le type de route censé refleter la qualité de la route, et une autre sans pondération, pour la longueur d'aménagements cyclables par ville.

In [78]:
highway_quality_mapping = {
    'service': 1,
    'track': 1,
    'path': 1,
    'trunk_link': 1,
    'motorway_link': 1,
    'residential': 2,
    'footway': 2,
    'cycleway': 2,
    'primary': 2,    
    'primary_link': 2,    
    'secondary': 3,
    'tertiary': 3,
    'secondary_link': 3,
    'tertiary_link': 3,
    'living_street': 3,
    'unclassified': 3,
    'pedestrian': 4,
    'steps': 4
}

In [79]:
df_a_velo['qual'] = df_a_velo['highway'].map(highway_quality_mapping)

In [80]:
quality_weights = {
    1: 1.25,
    2: 1.,
    3: 0.75,
    4: 0.5
}

In [81]:
# Nouvelle colonne pour la longueur pondérée

df_a_velo['longueur_pond'] = df_a_velo['longueur'] * df_a_velo['qual'].map(quality_weights)

In [83]:
total_longueur_pond = df_a_velo['longueur_pond'].sum()
print(total_longueur_pond) #longueur totale de l'échantillon avec majoration pondérée

13882578.0


In [89]:
total_longueur = df_a_velo['longueur'].sum()
print(total_longueur)

14174798


In [116]:
total_longueur_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur'].sum().reset_index()

In [117]:
total_longueur_pond_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond'].sum().reset_index()

In [118]:
df_amenagements = pd.merge(total_longueur_commune, total_longueur_pond_commune, on=['nom_com', 'insee_com'], suffixes=('_non_pond', '_pond'))

df_amenagements['difference'] = df_amenagements['longueur_pond'] - df_amenagements['longueur']

df_amenagements.sample(10)

,nom_com,insee_com,longueur,longueur_pond,difference
103,Boissy-Saint-Léger,94004,17246,19362.75,2116.75
944,Villeneuve-sous-Dammartin,77511,1038,951.50,-86.50
956,Villiers-Saint-Frédéric,78683,6677,6438.50,-238.50
628,Mériel,95392,9655,10348.50,693.50
897,Varennes-Jarcy,91631,6440,6235.00,-205.00
792,Saint-Martin-des-Champs,78565,381,302.50,-78.50
24,Achères-la-Forêt,77001,606,503.50,-102.50
787,Saint-Mammès,77419,1561,1333.50,-227.50
166,Chailly-en-Brie,77070,2467,2597.75,130.75
685,Othis,77349,4570,4758.50,188.50


Nous avons créé nos deux indicateurs par commune : longueur et longueur pondérée par commune, mais nous avons perdu de l'information dans cette opération puisque nous perdons de ce fait les coordonnées exactes des aménagements. Cependant, pour l'étude de la densité, cela pourra aussi s'avérer utile. Surtout nous avons besoin de ces indicateurs pour le travail économétrique que nous souhaitons mener, où nous considérons alors les communes comme des individus, et la longueur des pistes cyclables comme nous variable d'intérêt. Cependant, avant de passer à la modélisation économétrique pour tenter d'expliquer le développement des aménagements dans les différentes communes, nous souhaitons représenter et analyser spatialement la répartition des aménagements, leur densité (par habitant), leur densité (par kilomètre) et notamment compléter ces analyses grâce à la disponibilité des données du Vélib.

                   nom_com  longueur  longueur_pond  difference  \
0       10e Arrondissement     60281       54373.75    -5907.25   
1       11e Arrondissement     90116       84201.75    -5914.25   
2       12e Arrondissement    201748      199898.75    -1849.25   
3       13e Arrondissement    137731      128463.25    -9267.75   
4       14e Arrondissement    102618       94107.75    -8510.25   
...                    ...       ...            ...         ...   
1284           Évecquemont      6177        6162.00      -15.00   
1285                  Évry     81991       79291.00    -2700.00   
1286  Évry-Grégy-sur-Yerre      6997        5934.50    -1062.50   
1287  Évry-Grégy-sur-Yerre      6997        5934.50    -1062.50   
1288             Ézanville      2384        2220.75     -163.25   

                       nom   code  population    _score  
0                      NaN    NaN         NaN       NaN  
1                      NaN    NaN         NaN       NaN  
2                    

## Les vélibs en Ile de France

In [ ]:
url2 = "https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

In [ ]:
velib = gpd.read_file(url2)

In [ ]:
velib.sample(n=15)

## Données et ressources nécessaires pour l'étude spatiale

## Données socio-démographiques sur l'Ile de France

## Données socio-démographiques sur l'Ile de France

## Données complémentaires : accidents de la route, transports écologiques alternatifs (réseau ferré, bus)